In [1]:
import pandas as pd
import os
import glob

data_path = 'data/'

files = glob.glob(os.path.join(data_path, '*.csv'))

dataframes = []

for file in files:
    print(file)
    df = pd.read_csv(file, encoding='utf-8', encoding_errors=False)
    # df.head()

: 